# 🔎 VerificAI - Uma IA de suporte e análise de currículos!

## Instalações, importações e requisições necessárias para a execução correta do programa.

In [20]:
%pip -q install google-generativeai
%pip -q install google-adk
!pip install PyPDF2

import os
from google.colab import files, userdata
import PyPDF2
from google import genai
from IPython.display import display, Markdown
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
from datetime import date
import textwrap

os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

Conexão agente - parâmetro

In [21]:
def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])
    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
            for part in event.content.parts:
                if part.text:
                    final_response += part.text + "\n"
    return final_response.strip()


In [22]:
def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

Definições dos agentes e suas respectivas *características*

In [29]:
def agente_avaliador(curriculo_extraido: str):
    avaliador = Agent(
        name="agente_avaliador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um avaliador de currículos experiente, um especialista na área de recursos humanos. Avalie o conteúdo a seguir:
        Mantenha-se centrado nos pontos abaixo:
        1. Destaque pontos fortes.
        2. Aponte áreas de melhoria.
        3. Seja objetivo e profissional.
        """,
        description="Agente que avalia currículos criticamente."
    )
    entrada = f"Currículo:\n{curriculo_extraido}"
    return call_agent(avaliador, entrada)

In [24]:
def agente_otimizador(curriculo_extraido: str):
    otimizador = Agent(
        name="agente_otimizador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um especialista em revisão de currículos. Melhore a clareza, objetividade e impacto do conteúdo abaixo.
        Corrija erros gramaticais e melhore a linguagem profissional.
        """,
        description="Agente otimizador de currículo"
    )
    entrada = f"Currículo:\n{curriculo_extraido}"
    return call_agent(otimizador, entrada)

In [25]:
def agente_carta(curriculo_otimizado: str):
    redator = Agent(
        name="agente_carta",
        model="gemini-2.0-flash",
        instruction="""
        Você é um redator especializado em cartas de apresentação. Escreva uma carta em português profissional
        com base no currículo abaixo, destacando os pontos fortes do candidato.
        """,
        description="Agente que gera carta de apresentação"
    )
    entrada = f"Currículo:\n{curriculo_otimizado}"
    return call_agent(redator, entrada)

In [26]:
def agente_recomendador(curriculo_otimizado: str):
    recomendador = Agent(
        name="agente_recomendador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um mentor de carreira. Com base no currículo abaixo, recomende os próximos passos para o desenvolvimento profissional:
        - Habilidades a desenvolver
        - Ferramentas para aprender
        - Tipos de projeto para praticar
        - Recursos úteis (cursos, livros, etc)
        """,
        description="Agente de carreira para próximos passos"
    )
    entrada = f"Currículo:\n{curriculo_otimizado}"
    return call_agent(recomendador, entrada)

Execução principal

In [31]:
uploaded = files.upload()
file_name = list(uploaded.keys())[0]
pdf = open(file_name, 'rb')
reader = PyPDF2.PdfReader(pdf)
curriculo_extraido = "\n".join([page.extract_text() for page in reader.pages])
pdf.close()

print("🚀 Iniciando análise com os agentes da VerificAI...\n")

avaliacao = agente_avaliador(curriculo_extraido)
display(Markdown("## 📊 Avaliação Crítica"))
display(to_markdown(avaliacao))

otimizado = agente_otimizador(curriculo_extraido)
display(Markdown("## 📄 Currículo Otimizado"))
display(to_markdown(otimizado))

carta = agente_carta(otimizado)
display(Markdown("## ✉️ Carta de Apresentação"))
display(to_markdown(carta))

recomendacoes = agente_recomendador(otimizado)
display(Markdown("## 🧭 Próximos Passos"))
display(to_markdown(recomendacoes))

print("\n✅ Análise finalizada com sucesso!")


Saving MuriloSantos (2).pdf to MuriloSantos (2) (4).pdf
🚀 Iniciando análise com os agentes da VerificAI...



## 📊 Avaliação Crítica

> ## Avaliação do Currículo de Murilo Santos
> 
> **Pontos Fortes:**
> 
> *   **Objetivo Profissional:** O objetivo é claro e conciso, demonstrando foco no aprendizado e contribuição proativa.
> *   **Formação Acadêmica:** A formação em Análise e Desenvolvimento de Sistemas (FATEC e ETEC) é relevante para a área.
> *   **Habilidades Técnicas:** Demonstra conhecimento em diversas linguagens de programação (C#, JavaScript, Python, Java, SQL) e tecnologias (.NET, React).
> *   **Atividades Extracurriculares:** Experiência como voluntário demonstra proatividade, responsabilidade e habilidades de trabalho em equipe. A experiência na Igreja, com gerenciamento de dados e suporte a projetos, é valiosa.
> *   **Certificações:** As certificações CISCO, DIO e Alura indicam busca por aprimoramento constante e interesse em se manter atualizado com as tecnologias.
> *   **Soft Skills:** A lista de soft skills (Resiliência, Diligência, Interesse, Empatia) é relevante para o mercado de trabalho.
> *   **GitHub:** A inclusão do link para o GitHub permite que os recrutadores avaliem seus projetos e habilidades práticas.
> 
> **Áreas de Melhoria:**
> 
> *   **Formatação:** A formatação do currículo pode ser aprimorada para facilitar a leitura e destacar informações importantes.
> *   **Atividades Extracurriculares:** Detalhe mais as atividades voluntárias, quantificando resultados e destacando habilidades desenvolvidas.
> *   **Habilidades:**
>     *   **Nível de Inglês:** Seja mais específico sobre o nível de inglês (ex: leitura, escrita, conversação).
>     *   **Organização:** Agrupe as habilidades por categoria (ex: Linguagens de Programação, Ferramentas, Banco de Dados).
> *   **Experiência:** Considere incluir projetos pessoais relevantes ou projetos acadêmicos que demonstrem suas habilidades técnicas.
> *   **Objetivo:** Adapte o objetivo profissional para cada vaga específica, destacando as habilidades e experiências mais relevantes para a oportunidade.
> *   **Informações de Contato:** Verifique se as informações de contato estão corretas e atualizadas.
> *   **Revisão:** Revise o currículo cuidadosamente para evitar erros de gramática e ortografia.
> 
> **Recomendações:**
> 
> *   **Estrutura:** Organize o currículo em seções claras e concisas (ex: Informações Pessoais, Objetivo, Formação Acadêmica, Experiência, Habilidades, Certificações, Atividades Extracurriculares).
> *   **Palavras-chave:** Utilize palavras-chave relevantes para a área de atuação, com base nas descrições de vagas e nas tecnologias mais utilizadas no mercado.
> *   **Resultados:** Sempre que possível, quantifique seus resultados e conquistas em projetos e atividades.
> *   **Customização:** Adapte o currículo para cada vaga específica, destacando as habilidades e experiências mais relevantes para a oportunidade.
> *   **Portfólio:** Mantenha o GitHub atualizado com projetos relevantes e bem documentados.
> *   **Carta de Apresentação:** Considere incluir uma carta de apresentação personalizada para cada vaga, destacando seu interesse na empresa e suas qualificações mais relevantes.

## 📄 Currículo Otimizado

> **MURILO SANTOS**
> (19) 99900-1103 | murilosantosfvg@gmail.com | R. Carlos Roberto Ianses, 191 - Vinhedo, SP
> 
> **OBJETIVO**
> 
> Estudante de Análise e Desenvolvimento de Sistemas em busca de estágio para aplicar e aprimorar habilidades técnicas. Entusiasmado em contribuir proativamente em projetos desafiadores, consolidando conhecimentos e impulsionando o crescimento na área de tecnologia.
> 
> **FORMAÇÃO ACADÊMICA**
> 
> *   **Tecnólogo em Análise e Desenvolvimento de Sistemas**
>     FATEC Campinas | 2023 - Cursando
> *   **Técnico em Informática para Internet**
>     ETEC Vasco Antônio Venchiarutti | 2020 - 2022
> 
> **EXPERIÊNCIA VOLUNTÁRIA**
> 
> *   **Voluntário | Nanna Salgados** | 2023 - Dez/2024
>     *   Criação de artes visuais para campanhas de marketing.
>     *   Gerenciamento de campanhas de tráfego pago.
> *   **Staff Voluntário | Igreja** | 2022 – Presente
>     *   Recepção e integração de jovens.
>     *   Gerenciamento de planilhas de dados e informações.
>     *   Suporte na organização de projetos e eventos.
>     *   Participação em treinamento de liderança.
> 
> **HABILIDADES TÉCNICAS**
> 
> *   **Linguagens de Programação:** C# (.NET), JavaScript (React), Python, Java, SQL
> *   **Idiomas:** Português (Fluente), Inglês (Intermediário)
> 
> **CERTIFICAÇÕES**
> 
> *   Oracle Java Foundations | Janeiro de 2025
> *   Java e a arte da abstração - Dio | Fevereiro de 2025
> *   9° Imersão Dev - Alura | Abril de 2025
> *   Hardware Basics - CISCO | Maio de 2024
> *   Introdução ao .NET - DIO | Outubro de 2024
> *   Versionamento Git - DIO | Outubro de 2024
> 
> **SOFT SKILLS**
> 
> *   Resiliência
> *   Diligência
> *   Curiosidade
> *   Empatia
> 
> **LINKS ÚTEIS**
> 
> *   **GitHub:** [https://github.com/MiragemS](https://github.com/MiragemS)

## ✉️ Carta de Apresentação

> [Seu Nome]
> [Seu Endereço]
> [Seu Telefone]
> [Seu Email]
> 
> [Data]
> 
> [Nome do Contratante/Recursos Humanos]
> [Título do Contratante]
> [Nome da Empresa]
> [Endereço da Empresa]
> 
> **Assunto: Candidatura para Estágio em Análise e Desenvolvimento de Sistemas**
> 
> Prezado(a) [Nome do Contratante/Recursos Humanos],
> 
> Escrevo para expressar meu grande interesse na oportunidade de estágio em Análise e Desenvolvimento de Sistemas em [Nome da Empresa]. Como estudante do curso de Tecnólogo em Análise e Desenvolvimento de Sistemas na FATEC Campinas, estou ansioso para aplicar meus conhecimentos técnicos e contribuir ativamente para projetos desafiadores em sua empresa.
> 
> Minha formação como Técnico em Informática para Internet na ETEC Vasco Antônio Venchiarutti, aliada ao meu aprendizado contínuo na FATEC, me proporcionou uma base sólida em diversas linguagens de programação, incluindo C# (.NET), JavaScript (React), Python, Java e SQL. Além disso, possuo um bom nível de inglês, o que me permite acompanhar as últimas tendências e tecnologias da área.
> 
> Durante minha experiência como voluntário na Nanna Salgados, desenvolvi habilidades em criação de artes visuais e gerenciamento de campanhas de tráfego pago, demonstrando minha capacidade de aplicar meus conhecimentos técnicos em um contexto prático e de gerar resultados. Minha atuação como staff voluntário em minha igreja, onde gerencio planilhas de dados, organizo projetos e eventos, e participo de treinamento de liderança, reforça minha organização, proatividade e capacidade de trabalhar em equipe.
> 
> As certificações que obtive, como Oracle Java Foundations, Java e a arte da abstração (Dio), 9° Imersão Dev (Alura), Hardware Basics (CISCO), Introdução ao .NET (DIO) e Versionamento Git (DIO), atestam meu comprometimento com o aprendizado contínuo e o aprimoramento de minhas habilidades.
> 
> Sou um profissional resiliente, diligente, curioso e empático, sempre buscando aprender e me desenvolver. Acredito que minhas habilidades técnicas, experiência em projetos e soft skills me tornam um candidato ideal para contribuir de forma significativa para sua equipe.
> 
> Convido você a visitar meu GitHub ([https://github.com/MiragemS](https://github.com/MiragemS)) para conhecer alguns dos meus projetos.
> 
> Agradeço a sua atenção e tempo dedicado à minha candidatura. Ficarei muito feliz em discutir minhas qualificações em mais detalhes durante uma entrevista.
> 
> Atenciosamente,
> 
> Murilo Santos

## 🧭 Próximos Passos

> Murilo, seu currículo demonstra uma base sólida e um bom começo na área de tecnologia. Para te ajudar a dar os próximos passos na sua carreira, aqui estão algumas recomendações focadas no seu desenvolvimento profissional:
> 
> **Habilidades a Desenvolver:**
> 
> 1.  **Profundidade em Front-end:** Apesar de mencionar React, aprofunde-se em HTML, CSS e JavaScript (ES6+). Isso te dará mais controle e flexibilidade no desenvolvimento de interfaces.
> 2.  **Back-end com .NET:** Já que você tem conhecimento em C# e .NET, foque em aprender ASP.NET Core para criar APIs RESTful.
> 3.  **Banco de Dados:** Vá além do SQL básico. Explore modelagem de dados, otimização de queries e diferentes tipos de bancos de dados (relacionais e NoSQL).
> 4.  **Testes:** Aprenda sobre testes unitários, testes de integração e testes end-to-end. Testes garantem a qualidade do código e facilitam a manutenção.
> 5.  **Cloud Computing:** Familiarize-se com plataformas como AWS, Azure ou Google Cloud. Aprenda sobre serviços de computação, armazenamento e deployment.
> 
> **Ferramentas para Aprender:**
> 
> *   **Frameworks/Bibliotecas:**
>     *   **React Avançado:** Redux/Context API para gerenciamento de estado, Styled Components/Material UI para estilização.
>     *   **ASP.NET Core:** Entity Framework Core para ORM, Identity para autenticação e autorização.
> *   **Ferramentas de Desenvolvimento:**
>     *   **Visual Studio/VS Code:** Editores de código poderosos com muitos recursos e extensões.
>     *   **Docker:** Para conteinerização e ambientes de desenvolvimento consistentes.
>     *   **Postman/Insomnia:** Para testar APIs.
> *   **Ferramentas de CI/CD:**
>     *   **GitHub Actions:** Para automatizar builds, testes e deployments.
>     *   **Jenkins:** Uma ferramenta de automação open-source popular.
> 
> **Tipos de Projeto para Praticar:**
> 
> 1.  **Clone de Interfaces Populares:**
>     *   Escolha um site ou app popular (ex: Twitter, Instagram) e tente replicar a interface usando React e Styled Components.
> 2.  **API RESTful com .NET:**
>     *   Crie uma API para um sistema de gerenciamento de tarefas (To-Do List) ou um catálogo de produtos.
> 3.  **Aplicativo Full-Stack:**
>     *   Combine o front-end (React) e o back-end (.NET) para criar um aplicativo completo.
>     *   Exemplo: Um sistema de gerenciamento de biblioteca com autenticação, listagem de livros, cadastro de novos livros, etc.
> 4.  **Projetos Open Source:**
>     *   Contribua para projetos open source no GitHub. Isso te dará experiência em trabalhar em equipe e lidar com código de outras pessoas.
> 
> **Recursos Úteis:**
> 
> *   **Cursos Online:**
>     *   **Alura:** Cursos completos de front-end, back-end e cloud.
>     *   **Udemy:** Grande variedade de cursos com preços acessíveis.
>     *   **Coursera/edX:** Cursos de universidades e empresas renomadas.
>     *   **Rocketseat:** Foco em JavaScript, React e Node.js.
>     *   **Microsoft Learn:** Conteúdo oficial da Microsoft sobre .NET e Azure.
> *   **Livros:**
>     *   **"Clean Code"** de Robert C. Martin: Boas práticas de programação.
>     *   **"Design Patterns"** de Erich Gamma et al.: Padrões de projeto para resolver problemas comuns.
>     *   **"Refactoring"** de Martin Fowler: Técnicas para melhorar o código existente.
> *   **Comunidades:**
>     *   **Stack Overflow:** Para tirar dúvidas e encontrar soluções.
>     *   **Meetups:** Participe de eventos locais para conhecer outros desenvolvedores.
>     *   **Grupos no Telegram/Discord:** Troque ideias e receba ajuda da comunidade.
> *   **Documentação:**
>     *   **MDN Web Docs:** Referência completa de HTML, CSS e JavaScript.
>     *   **Documentação do React:** Aprenda sobre os recursos e APIs do React.
>     *   **Documentação do .NET:** Guia completo da plataforma .NET.
> 
> **Próximos Passos:**
> 
> 1.  **Defina um Plano de Estudos:**
>     *   Escolha uma ou duas áreas para focar (ex: front-end e .NET).
>     *   Crie um cronograma de estudos com metas semanais.
> 2.  **Crie um Portfólio:**
>     *   Adicione seus projetos no GitHub.
>     *   Crie um site pessoal para mostrar seus projetos e habilidades.
> 3.  **Networking:**
>     *   Participe de eventos da área.
>     *   Conecte-se com outros desenvolvedores no LinkedIn.
> 4.  **Procure Oportunidades de Estágio:**
>     *   Atualize seu currículo com as novas habilidades e projetos.
>     *   Candidate-se para vagas de estágio em empresas que utilizam as tecnologias que você está aprendendo.
>     *   Prepare-se para entrevistas técnicas.
> 
> Lembre-se que o aprendizado é contínuo. Mantenha-se atualizado com as novidades da área e continue praticando. Com dedicação e foco, você alcançará seus objetivos!


✅ Análise finalizada com sucesso!
